# Extracting Data From the WMO app

## 1 Insturactions of the WMO

Use the following steps to carry out the project
- Use this link (https://community.wmo.int/en/members/profiles to extract data on
heatwaves warning system/services, see example below, if heatwave is in the list of
warnings provided, then indicate a yes in the provided excel sheet (for the corresponding
country in column B in the excel sheet called “Heatwaves Warning Systems (WMO data)”).


full screenmode : https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9


## 2  Required Libraries

In [53]:
#The webdriver module in Selenium allows for automated control of web browsers.
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


## 3 Opening up the web browser and printing a sample text

The function below opens up chrome browser and waits for 60 minutes to load the data 

input: The website link

Output : the driver and wabdriverwait instance

In [54]:

def openChrome(link):
    # Options customize Chrome WebDriver behavior
    options = Options()

    # Initialize the WebDriver (e.g., Chrome)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Navigate to the desired URL
    driver.get(link)

    # wait for page to load
    wait = WebDriverWait(driver, 60)

    return driver, wait

In [55]:
link = "https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9"
driver_instance, wait  = openChrome(link)

Using the driver instance that we created print an sample text

In [56]:
# Locate all div elements with class "textbox"
div_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="textbox"]')))

# Iterate over the list of elements and print the text content of each
for div_element in div_elements:
    print(div_element.text)
    print("-----")  # Just a separator for clarity

# driver_instance.quit()

Member Profiles
-----
The information contained in these profiles have been provided by WMO Members. The information is based on a self-assessment exercise. The date displayed in each section corresponds with the date in which the information was provided to WMO. 

If you find any issue or inconsistency with the information displayed, please contact community@wmo.int.
-----
Permanent Representative:
-----
Hydrological Adviser:
-----
National Meteorological Service:
-----
Responsible Ministry:
-----


## 4. Clicking on the Early Warning Services 

In [67]:
#EC stands for "Expected Conditions." It is a set of predefined conditions that the Selenium WebDriver uses.
#presence_of_element_located is one of many expected conditions (EC). It checks if an element is present on the DOM of a page.
def wait_and_click(driver, wait, css_selector):
    element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    driver.execute_script("arguments[0].click();", element)

In [68]:
css_selector = 'a[title=" Early Warning Services"]'
wait_and_click(driver_instance, wait, css_selector)

## 5 - Get Warnings List

In [69]:
def saveWarnings(driver, css_selector):
    #'.title' is the class name of the element that contains the warning text
    warnings= driver_instance.find_elements(By.CSS_SELECTOR, css_selector)
    return warnings

warnings = saveWarnings(driver_instance, '.title')


In [70]:

for warning in warnings:
    print(warning.text)

Dust storm/Sandstorm
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
Yes
2 jours
Yes
Yes
No
Yes
Yes


## 6 getting all country names 

In [111]:

def getCountryNames(driver, wait):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    loaded_countries = set()
    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll to the last element to load more countries
        ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()
    print(loaded_countries)
    # Close the dropdown menu by clicking the dropdown button again
    dropdown.click()
    return loaded_countries

In [112]:
loaded_countries = getCountryNames(driver_instance, wait)

{'Thailand', 'Ethiopia', 'Belarus', 'Malaysia', 'Finland', 'Latvia', 'Costa Rica', 'Norway', 'Macao, China', 'Hong Kong, China', 'Vanuatu', 'Myanmar', 'Romania', 'Uganda', 'Republic of Moldova', 'Zambia', 'Cyprus', 'Niger', 'Chile', 'Republic of Korea', 'Ukraine', 'Curaçao and Sint Maarten', 'China', 'Sao Tome and Principe', 'Mauritania', 'France', 'Gabon', 'Seychelles', 'Azerbaijan', 'Sweden', 'Comoros', 'Kuwait', 'Burundi', 'Tajikistan', 'Guatemala', 'Poland', 'Democratic Republic of the Congo', 'Nauru', 'Panama', 'Argentina', 'Bosnia and Herzegovina', 'Dominica', 'Oman', 'Hungary', 'British Caribbean Territories - Anguilla', 'Pakistan', 'Iraq', 'Serbia', 'Bhutan', 'Guinea-Bissau', 'Bahrain', 'Chad', 'Zimbabwe', "Democratic People's Republic of Korea", 'Cuba', 'Cameroon', 'Antigua and Barbuda', 'Netherlands', 'Greece', 'Indonesia', 'Albania', 'Turkmenistan', 'Brazil', 'Guyana', 'Venezuela, Bolivarian Republic of', "Côte d'Ivoire", 'Trinidad and Tobago', 'Qatar', 'United Arab Emirates

In [63]:
print(len(loaded_countries ))

198


## 7 Selecting A country

In [71]:
def select_country_from_dropdown(driver, wait, target_country):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    # Clear the search input
    search_input = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.searchInput'))
    )
    search_input.clear()

    loaded_countries = set()
    country_selected = False

    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if target country is in the current batch of loaded countries
        if target_country in current_countries:
            for option in current_countries_elements:
                if option.text == target_country:
                    option.click()
                    country_selected = True
                    break

        # If the country has been selected, break out of the loop
        if country_selected:
            break
        
        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll to the last element to load more countries
        ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()

    return country_selected


In [40]:
success = select_country_from_dropdown(driver_instance, wait, "United States of America")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=117.0.5938.92)
Stacktrace:
0   chromedriver                        0x0000000108eaced8 chromedriver + 5091032
1   chromedriver                        0x0000000108ea3c43 chromedriver + 5053507
2   chromedriver                        0x0000000108a3f366 chromedriver + 447334
3   chromedriver                        0x0000000108a83fd1 chromedriver + 729041
4   chromedriver                        0x0000000108ab5662 chromedriver + 931426
5   chromedriver                        0x0000000108a81ba8 chromedriver + 719784
6   chromedriver                        0x0000000108ab581e chromedriver + 931870
7   chromedriver                        0x0000000108ad16e1 chromedriver + 1046241
8   chromedriver                        0x0000000108ab5433 chromedriver + 930867
9   chromedriver                        0x0000000108a80042 chromedriver + 712770
10  chromedriver                        0x0000000108a8126e chromedriv

In [ ]:
success = select_country_from_dropdown(driver_instance, wait, "India")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

In [ ]:

warnings = saveWarnings(driver_instance, '.title')

for warning in warnings:
    print(warning.text)

In [136]:
success = select_country_from_dropdown(driver_instance, wait, "Argentina")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=117.0.5938.92)
Stacktrace:
0   chromedriver                        0x000000010e86ced8 chromedriver + 5091032
1   chromedriver                        0x000000010e863c43 chromedriver + 5053507
2   chromedriver                        0x000000010e3ff366 chromedriver + 447334
3   chromedriver                        0x000000010e443fd1 chromedriver + 729041
4   chromedriver                        0x000000010e475662 chromedriver + 931426
5   chromedriver                        0x000000010e441ba8 chromedriver + 719784
6   chromedriver                        0x000000010e47581e chromedriver + 931870
7   chromedriver                        0x000000010e4916e1 chromedriver + 1046241
8   chromedriver                        0x000000010e475433 chromedriver + 930867
9   chromedriver                        0x000000010e440042 chromedriver + 712770
10  chromedriver                        0x000000010e44126e chromedriv

In [72]:
warnings = saveWarnings(driver_instance, '.title')

for warning in warnings:
    print(warning.text)

Dust storm/Sandstorm
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
Yes
2 jours
Yes
Yes
No
Yes
Yes


## selecting 8 country and getting the information

1- Initiate global variables

    - initiate a dictinary

    - driver, wait

2- Load all the country names

3 -Click on Early warning systems

----- repeat this 8 times ----

4- Click on the desired country name

5- extract warning systems in a list

   (if the list is empty)
   
6- save country name and the list in the map

In [ ]:
# Convert the set to a list
countries_list = list(loaded_countries)

# # Sort the list
# countries_list.sort()

# print(countries_list)
# test= countries_list[0:8]

In [ ]:
size = len(countries_list)//20
slice_list = [countries_list[i:i + size] for i in range(0, len(countries_list), size)]

for slice in slice_list:
    print(slice)

['Papua New Guinea', 'Singapore', 'Sao Tome and Principe', 'Czechia', 'Djibouti', 'Sri Lanka', 'Burkina Faso', 'Brunei Darussalam', 'British Caribbean Territories - Montserrat']
['Nicaragua', 'Brazil', 'Kuwait', 'Ireland', 'Poland', 'Pakistan', 'Bolivia (Plurinational State of)', 'North Macedonia', 'Tuvalu']
['Morocco', 'Kiribati', 'Seychelles', 'Uzbekistan', 'Egypt', 'Curaçao and Sint Maarten', "Democratic People's Republic of Korea", 'Chad', 'Azerbaijan']
['Chile', 'Zambia', 'British Caribbean Territories - Anguilla', 'Latvia', 'Panama', 'Ukraine', 'Macao, China', 'Antigua and Barbuda', 'Saint Lucia']
['Cameroon', 'Croatia', 'Hong Kong, China', 'British Caribbean Territories', 'Malta', 'Kazakhstan', 'United States of America', 'Norway', 'Madagascar']
['Georgia', 'Thailand', 'Cambodia', 'Guyana', 'Afghanistan', 'Bangladesh', 'Iraq', 'Slovakia', 'Costa Rica']
['Ghana', 'Algeria', "Côte d'Ivoire", 'United Arab Emirates', 'Albania', 'Tajikistan', 'Mongolia', 'Eritrea', "Lao People's Demo

In [73]:
import pickle

# # Your list of arrays
# arrays = slice_list

# # Save to a file
# with open('arrays.pkl', 'wb') as f:
#     pickle.dump(arrays, f)



In [74]:
# # Load from a file
with open('arrays.pkl', 'rb') as f:
    loaded_arrays = pickle.load(f)

we can load this list and acces them so that the country list does not change

In [75]:
loaded_arrays[5]

['Georgia',
 'Thailand',
 'Cambodia',
 'Guyana',
 'Afghanistan',
 'Bangladesh',
 'Iraq',
 'Slovakia',
 'Costa Rica']

lets divide our country list in to 10 arrays



In [81]:
slice_list = loaded_arrays


In [82]:
loaded_arrays[5]

['Georgia',
 'Thailand',
 'Cambodia',
 'Guyana',
 'Afghanistan',
 'Bangladesh',
 'Iraq',
 'Slovakia',
 'Costa Rica']

In [ ]:
slice_list[5]

['Georgia',
 'Thailand',
 'Cambodia',
 'Guyana',
 'Afghanistan',
 'Bangladesh',
 'Iraq',
 'Slovakia',
 'Costa Rica']

In [ ]:
# test_list = ['Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco']
# print(len(test_list))
my_dict = {}

possible fix 

In [ ]:
for country in slice_list[0]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)
        


Papua New Guinea
country been selected successfully!
No warnings found for this country.
-----
Singapore
country been selected successfully!
Drought/Dry spell
Haze/Smoke
Heat wave
High Seas/Rogue waves etc.
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Tropical cyclone
Volcanic ash
No
No
No
Yes
Yes
Yes
Yes
Yes
All
Drought/Dry spell
Haze/Smoke
Heat wave
High Seas/Rogue waves etc.
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Tropical cyclone
Volcanic ash
No
No
No
Yes
Yes
Yes
Yes
Yes
All
-----
Sao Tome and Principe
country been selected successfully!
No warnings found for this country.
-----
Czechia
country been selected successfully!
Cold wave
Flood
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
100000
Cold wave
Flood
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Fo

In [ ]:
my_dict

{'Papua New Guinea': [0],
 'Singapore': ['Drought/Dry spell',
  'Haze/Smoke',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/Squall lines',
  'Tropical cyclone',
  'Volcanic ash',
  'No',
  'No',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'All'],
 'Sao Tome and Principe': [0],
 'Czechia': ['Cold wave',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  '100000'],
 'Djibouti': ['Dust storm/Sandstorm',
  'Rain/Wet Spell',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '6HEURES',
  'Yes',
  'Yes',
  'Yes'],
 'Sri Lanka': [0],
 'Burkina Faso': ['Cold wave',
  'Drought/Dry spell',
  'Dust storm/Sandstorm',
  'Haze/Smoke',
  'Heat wave',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/S

array 2 , dictinary 2

In [ ]:
my_dict_2 = {}

In [ ]:
for country in slice_list[1]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_2[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_2[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)
        


Nicaragua
country been selected successfully!
No warnings found for this country.
-----
Brazil
country been selected successfully!
Cold wave
Drought/Dry spell
Flood
Fog
Frost
Hail
Haze/Smoke
Heat wave
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Wind
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
All of Brazilian territory is covered by Early Warnings
Cold wave
Drought/Dry spell
Flood
Fog
Frost
Hail
Haze/Smoke
Heat wave
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Wind
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
All of Brazilian territory is covered by Early Warnings
-----
Kuwait
country been selected successfully!
No warnings found for this country.
-----
Ireland
country been selected successfully!
Cold wave
Fog
Freezing rain
Hail
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wind
Yes
Yes
Yes
2-days
Yes
Yes
No
No
100000
Cold wave
Fog
Freezing rain
Hail
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing


In [ ]:
my_dict_2

{'Nicaragua': [0],
 'Brazil': ['Cold wave',
  'Drought/Dry spell',
  'Flood',
  'Fog',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/Squall lines',
  'Wind',
  'No',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'All of Brazilian territory is covered by Early Warnings'],
 'Kuwait': [0],
 'Ireland': ['Cold wave',
  'Fog',
  'Freezing rain',
  'Hail',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'High UV radiation',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '2-days',
  'Yes',
  'Yes',
  'No',
  'No',
  '100000'],
 'Poland': ['Cold wave',
  'Drought/Dry spell',
  'Extra-tropical cyclone',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Yes',
  'No',
  'Yes',
  'up to 4 day

array 3

In [ ]:
my_dict_3 = {}

In [ ]:
for item in range(2,4):
    print(item)

2
3


this code run

In [ ]:
for country in slice_list[2]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_3[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_3[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)



In [ ]:
my_dict_3

{'Morocco': ['Cold wave',
  'Dust storm/Sandstorm',
  'Hail',
  'Heat wave',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wind',
  'Yes',
  'No',
  '3 - jours',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes'],
 'Kiribati': [0],
 'Seychelles': ['Drought/Dry spell',
  'Flood',
  'Lightning',
  'Storm surge/Coastal flood',
  'Tropical cyclone',
  'Tsunami',
  'Wind',
  'Yes',
  'No',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'No'],
 'Uzbekistan': [0],
 'Egypt': ['Cold wave',
  'Drought/Dry spell',
  'Dust storm/Sandstorm',
  'Flood',
  'Fog',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'Lightning',
  'Pollen pollution/Polluted air',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Wind',
  'Yes',
  'No',
  'Yes',
  'days or 6',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes'],
 'Curaçao and Sint Maarten': [0],
 "Democratic People's Republic of Korea": [0],
 'Chad': ['Drought/Dry spell',
  'Dust storm/Sandstorm'

array 4

In [ ]:
my_dict_4 = {}

In [ ]:
for country in slice_list[3]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_4[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_4[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)



Chile
country been selected successfully!
Cold wave
Drought/Dry spell
Fog
Frost
Hail
Haze/Smoke
Heat wave
High UV radiation
Icing
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Tornado
Volcanic ash
Yes
No
Yes
5 días
Yes
Yes
Yes
Yes
Yes
No
Cold wave
Drought/Dry spell
Fog
Frost
Hail
Haze/Smoke
Heat wave
High UV radiation
Icing
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Tornado
Volcanic ash
Yes
No
Yes
5 días
Yes
Yes
Yes
Yes
Yes
No
-----
Zambia
country been selected successfully!
No warnings found for this country.
-----
British Caribbean Territories - Anguilla
country been selected successfully!
No warnings found for this country.
-----
Latvia
country been selected successfully!
No warnings found for this country.
-----
Panama
country been selected successfully!
Drought/Dry spell
Flood
High UV radiation
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Tropical cyclone
Wild land fire/Forest fire
Wind
Yes
No
Yes
Aviso de Vigilancia son 72 horas de anticipación y de

In [ ]:
my_dict_4

{'Chile': ['Cold wave',
  'Drought/Dry spell',
  'Fog',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'High UV radiation',
  'Icing',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Tornado',
  'Volcanic ash',
  'Yes',
  'No',
  'Yes',
  '5 días',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'No'],
 'Zambia': [0],
 'British Caribbean Territories - Anguilla': [0],
 'Latvia': [0],
 'Panama': ['Drought/Dry spell',
  'Flood',
  'High UV radiation',
  'Lightning',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Tropical cyclone',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'No',
  'Yes',
  'Aviso de Vigilancia son 72 horas de anticipación y de Advertencia de dos a tres horas de anticipación y se actualiza cada tres horas.',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'No',
  'El 75 de la población.'],
 'Ukraine': ['Avalanche',
  'Cold wave',
  'Drought/Dry spell',
  'Dust storm/Sandstorm',
  'Extra-tropical cyclone',
  'Flood',
  'Fog',
  'F

array 5

In [ ]:
my_dict_5 = {}

In [ ]:
for country in slice_list[4]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_5[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_5[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)



Cameroon
country been selected successfully!
Cold wave
Dust storm/Sandstorm
Fog
Haze/Smoke
Heat wave
Landslide/Mudslide & Debris flow
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Wind
No
Yes
Yes
72 heures
Yes
Yes
No
Cold wave
Dust storm/Sandstorm
Fog
Haze/Smoke
Heat wave
Landslide/Mudslide & Debris flow
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Wind
No
Yes
Yes
72 heures
Yes
Yes
No
-----
Croatia
country been selected successfully!
Cold wave
Drought/Dry spell
Flood
Fog
Freezing rain
Frost
Hail
Haze/Smoke
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Lightning
Pollen pollution/Polluted air
Rain/Wet Spell
Yes
Yes
Yes
2 days
Yes
Yes
Yes
Yes
Yes
Yes
100
Cold wave
Drought/Dry spell
Flood
Fog
Freezing rain
Frost
Hail
Haze/Smoke
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Lightning
Pollen pollution/Polluted air
Rain/Wet Spell
Yes
Yes
Yes
2 days
Yes
Yes
Yes
Yes
Yes
Yes
100
-----
Hong Kong, China
country been selecte

In [ ]:
my_dict_5

{'Cameroon': ['Cold wave',
  'Dust storm/Sandstorm',
  'Fog',
  'Haze/Smoke',
  'Heat wave',
  'Landslide/Mudslide & Debris flow',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Wind',
  'No',
  'Yes',
  'Yes',
  '72 heures',
  'Yes',
  'Yes',
  'No'],
 'Croatia': ['Cold wave',
  'Drought/Dry spell',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'High UV radiation',
  'Icing',
  'Lightning',
  'Pollen pollution/Polluted air',
  'Rain/Wet Spell',
  'Yes',
  'Yes',
  'Yes',
  '2 days',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  '100'],
 'Hong Kong, China': ['Cold wave',
  'Dust storm/Sandstorm',
  'Flood',
  'Frost',
  'Hail',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'High UV radiation',
  'Icing',
  'Landslide/Mudslide & Debris flow',
  'Lightning',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Tornado',
  'Yes

In [ ]:
slice_list[5]

['Georgia',
 'Thailand',
 'Cambodia',
 'Guyana',
 'Afghanistan',
 'Bangladesh',
 'Iraq',
 'Slovakia',
 'Costa Rica']

save the progress, there has been 5 dictionary 
- combine them
- add dates 
- save as a csv file 

In [ ]:
import pandas as pd

In [ ]:
my_dict_2

{'Nicaragua': [0],
 'Brazil': ['Cold wave',
  'Drought/Dry spell',
  'Flood',
  'Fog',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/Squall lines',
  'Wind',
  'No',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'All of Brazilian territory is covered by Early Warnings'],
 'Kuwait': [0],
 'Ireland': ['Cold wave',
  'Fog',
  'Freezing rain',
  'Hail',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'High UV radiation',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '2-days',
  'Yes',
  'Yes',
  'No',
  'No',
  '100000'],
 'Poland': ['Cold wave',
  'Drought/Dry spell',
  'Extra-tropical cyclone',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Yes',
  'No',
  'Yes',
  'up to 4 day

In [ ]:
combined_dict = {**my_dict, **my_dict_2, **my_dict_3, **my_dict_4, **my_dict_5}

combined_dict

{'Papua New Guinea': [0],
 'Singapore': ['Drought/Dry spell',
  'Haze/Smoke',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/Squall lines',
  'Tropical cyclone',
  'Volcanic ash',
  'No',
  'No',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'All'],
 'Sao Tome and Principe': [0],
 'Czechia': ['Cold wave',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  '100000'],
 'Djibouti': ['Dust storm/Sandstorm',
  'Rain/Wet Spell',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '6HEURES',
  'Yes',
  'Yes',
  'Yes'],
 'Sri Lanka': [0],
 'Burkina Faso': ['Cold wave',
  'Drought/Dry spell',
  'Dust storm/Sandstorm',
  'Haze/Smoke',
  'Heat wave',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/S

In [ ]:
countries, warnings = zip(*combined_dict.items())
df = pd.DataFrame({'Country': countries, 'Warnings': warnings})
df

,Country,Warnings
0,Papua New Guinea,[0]
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig..."
2,Sao Tome and Principe,[0]
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ..."
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y..."
5,Sri Lanka,[0]
6,Burkina Faso,"[Cold wave, Drought/Dry spell, Dust storm/Sand..."
7,Brunei Darussalam,[0]
8,British Caribbean Territories - Montserrat,[0]
9,Nicaragua,[0]


adding dates

In [ ]:
df["date"]= "09/26/2023"

df

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
5,Sri Lanka,[0],09/26/2023
6,Burkina Faso,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/26/2023
7,Brunei Darussalam,[0],09/26/2023
8,British Caribbean Territories - Montserrat,[0],09/26/2023
9,Nicaragua,[0],09/26/2023


save as a pickle

In [ ]:
# # # Save to a file
# # Save DataFrame to a pickle file
# df.to_pickle('dataframe.pkl')

In [77]:
import pandas as pd

In [78]:
df_loaded = pd.read_pickle('dataframe.pkl')

where we left!!! 

In [49]:
my_dict_6 = {}

In [50]:
slice_list[5]

['Georgia',
 'Thailand',
 'Cambodia',
 'Guyana',
 'Afghanistan',
 'Bangladesh',
 'Iraq',
 'Slovakia',
 'Costa Rica']

In [51]:
for country in slice_list[5]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_6[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_6[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)



Georgia
country been selected successfully!
No warnings found for this country.
-----
Thailand
country been selected successfully!
Drought/Dry spell
Flood
Hail
Haze/Smoke
High Seas/Rogue waves etc.
High UV radiation
Icing
Landslide/Mudslide & Debris flow
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tropical cyclone
Tsunami
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
N/A
Drought/Dry spell
Flood
Hail
Haze/Smoke
High Seas/Rogue waves etc.
High UV radiation
Icing
Landslide/Mudslide & Debris flow
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tropical cyclone
Tsunami
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
N/A
-----
Cambodia
country been selected successfully!
Drought/Dry spell
Flood
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tropical cyclone
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
No
It varies depending on the territory. There is no uniform answer to this question but the urban areas have more people at 

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=117.0.5938.92)
Stacktrace:
0   chromedriver                        0x00000001049c4ed8 chromedriver + 5091032
1   chromedriver                        0x00000001049bbc43 chromedriver + 5053507
2   chromedriver                        0x0000000104557366 chromedriver + 447334
3   chromedriver                        0x000000010459bfd1 chromedriver + 729041
4   chromedriver                        0x00000001045cd662 chromedriver + 931426
5   chromedriver                        0x0000000104599ba8 chromedriver + 719784
6   chromedriver                        0x00000001045cd81e chromedriver + 931870
7   chromedriver                        0x00000001045e96e1 chromedriver + 1046241
8   chromedriver                        0x00000001045cd433 chromedriver + 930867
9   chromedriver                        0x0000000104598042 chromedriver + 712770
10  chromedriver                        0x000000010459926e chromedriv

In [52]:
slice_list[5]

['Georgia',
 'Thailand',
 'Cambodia',
 'Guyana',
 'Afghanistan',
 'Bangladesh',
 'Iraq',
 'Slovakia',
 'Costa Rica']

missing 3 country iraq , slovakia, costa ric   but we can save those in a different dictinary

In [115]:
my_dict_6

{'Georgia': [0],
 'Thailand': ['Drought/Dry spell',
  'Flood',
  'Hail',
  'Haze/Smoke',
  'High Seas/Rogue waves etc.',
  'High UV radiation',
  'Icing',
  'Landslide/Mudslide & Debris flow',
  'Lightning',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Tropical cyclone',
  'Tsunami',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'N/A'],
 'Cambodia': ['Drought/Dry spell',
  'Flood',
  'Lightning',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Tropical cyclone',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'It varies depending on the territory. There is no uniform answer to this question but the urban areas have more people at risk that in the coastal area. However, the elements at risk cover not only human but productive assets.'],
 'Guyana': ['Drought/Dry spell',
  'Flood',
  'Fog',
  'Heat wave',
  'High Seas/Rogue waves et

In [116]:
countries, warnings = zip(*my_dict_6.items())
df2 = pd.DataFrame({'Country': countries, 'Warnings': warnings})
df2

,Country,Warnings
0,Georgia,[0]
1,Thailand,"[Drought/Dry spell, Flood, Hail, Haze/Smoke, H..."
2,Cambodia,"[Drought/Dry spell, Flood, Lightning, Rain/Wet..."
3,Guyana,"[Drought/Dry spell, Flood, Fog, Heat wave, Hig..."
4,Afghanistan,[0]
5,Bangladesh,[0]


In [118]:
df2["date"]= "09/27/2023"
df2

,Country,Warnings,date
0,Georgia,[0],09/27/2023
1,Thailand,"[Drought/Dry spell, Flood, Hail, Haze/Smoke, H...",09/27/2023
2,Cambodia,"[Drought/Dry spell, Flood, Lightning, Rain/Wet...",09/27/2023
3,Guyana,"[Drought/Dry spell, Flood, Fog, Heat wave, Hig...",09/27/2023
4,Afghanistan,[0],09/27/2023
5,Bangladesh,[0],09/27/2023


In [79]:
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
5,Sri Lanka,[0],09/26/2023
6,Burkina Faso,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/26/2023
7,Brunei Darussalam,[0],09/26/2023
8,British Caribbean Territories - Montserrat,[0],09/26/2023
9,Nicaragua,[0],09/26/2023


In [127]:
type(df_loaded)

pandas.core.frame.DataFrame

In [132]:
df_loaded = pd.concat([df_loaded, df2], ignore_index=True)

In [133]:
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
5,Sri Lanka,[0],09/26/2023
6,Burkina Faso,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/26/2023
7,Brunei Darussalam,[0],09/26/2023
8,British Caribbean Territories - Montserrat,[0],09/26/2023
9,Nicaragua,[0],09/26/2023


In [137]:
df_loaded.to_pickle('dataframe.pkl')
df_loaded = pd.read_pickle('dataframe.pkl')
df_loaded

In [138]:
df_loaded = pd.read_pickle('dataframe.pkl')
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
5,Sri Lanka,[0],09/26/2023
6,Burkina Faso,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/26/2023
7,Brunei Darussalam,[0],09/26/2023
8,British Caribbean Territories - Montserrat,[0],09/26/2023
9,Nicaragua,[0],09/26/2023


In [84]:
my_dict_7 = {}

In [85]:
slice_list[6]

['Ghana',
 'Algeria',
 "Côte d'Ivoire",
 'United Arab Emirates',
 'Albania',
 'Tajikistan',
 'Mongolia',
 'Eritrea',
 "Lao People's Democratic Republic"]

In [87]:
for country in slice_list[6]:
    print(country)
    time.sleep(30)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(30)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_7[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_7[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(30)



Ghana
country been selected successfully!
Drought/Dry spell
Dust storm/Sandstorm
Flood
Haze/Smoke
High Seas/Rogue waves etc.
Lightning
Pollen pollution/Polluted air
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
No
Yes
Yes
Yes
Yes
No
people who have assess to smartphones immediately receive while others wait for the radio and television dessemination
Drought/Dry spell
Dust storm/Sandstorm
Flood
Haze/Smoke
High Seas/Rogue waves etc.
Lightning
Pollen pollution/Polluted air
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
No
Yes
Yes
Yes
Yes
No
people who have assess to smartphones immediately receive while others wait for the radio and television dessemination
-----
Algeria
country been selected successfully!
Dust storm/Sandstorm
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind

In [88]:
my_dict_7

{'Ghana': ['Drought/Dry spell',
  'Dust storm/Sandstorm',
  'Flood',
  'Haze/Smoke',
  'High Seas/Rogue waves etc.',
  'Lightning',
  'Pollen pollution/Polluted air',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'people who have assess to smartphones immediately receive while others wait for the radio and television dessemination'],
 'Algeria': ['Dust storm/Sandstorm',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '2 jours',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes'],
 "Côte d'Ivoire": ['Drought/Dry spell',
  'Dust storm/Sandstorm',
  'Flood',
  'Haze/Smoke',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'Landslide/Mudslide & Debris flow',
  'L

In [89]:
countries, warnings = zip(*my_dict_7.items())
df3 = pd.DataFrame({'Country': countries, 'Warnings': warnings})
df3

,Country,Warnings
0,Ghana,"[Drought/Dry spell, Dust storm/Sandstorm, Floo..."
1,Algeria,"[Dust storm/Sandstorm, Fog, Freezing rain, Fro..."
2,Côte d'Ivoire,"[Drought/Dry spell, Dust storm/Sandstorm, Floo..."
3,United Arab Emirates,"[Dust storm/Sandstorm, Flood, Freezing rain, H..."
4,Albania,[0]
5,Tajikistan,[0]
6,Mongolia,"[Cold wave, Dust storm/Sandstorm, Flood, Freez..."
7,Eritrea,[0]
8,Lao People's Democratic Republic,"[Drought/Dry spell, Flood, Landslide/Mudslide ..."


In [90]:
df3["date"]= "09/27/2023"
df3

,Country,Warnings,date
0,Ghana,"[Drought/Dry spell, Dust storm/Sandstorm, Floo...",09/27/2023
1,Algeria,"[Dust storm/Sandstorm, Fog, Freezing rain, Fro...",09/27/2023
2,Côte d'Ivoire,"[Drought/Dry spell, Dust storm/Sandstorm, Floo...",09/27/2023
3,United Arab Emirates,"[Dust storm/Sandstorm, Flood, Freezing rain, H...",09/27/2023
4,Albania,[0],09/27/2023
5,Tajikistan,[0],09/27/2023
6,Mongolia,"[Cold wave, Dust storm/Sandstorm, Flood, Freez...",09/27/2023
7,Eritrea,[0],09/27/2023
8,Lao People's Democratic Republic,"[Drought/Dry spell, Flood, Landslide/Mudslide ...",09/27/2023


In [91]:
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
5,Sri Lanka,[0],09/26/2023
6,Burkina Faso,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/26/2023
7,Brunei Darussalam,[0],09/26/2023
8,British Caribbean Territories - Montserrat,[0],09/26/2023
9,Nicaragua,[0],09/26/2023


In [92]:
df_loaded = pd.concat([df_loaded, df3], ignore_index=True)
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
5,Sri Lanka,[0],09/26/2023
6,Burkina Faso,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/26/2023
7,Brunei Darussalam,[0],09/26/2023
8,British Caribbean Territories - Montserrat,[0],09/26/2023
9,Nicaragua,[0],09/26/2023


In [94]:
df_loaded.to_pickle('dataframe.pkl')
df_loaded = pd.read_pickle('dataframe.pkl')
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
5,Sri Lanka,[0],09/26/2023
6,Burkina Faso,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/26/2023
7,Brunei Darussalam,[0],09/26/2023
8,British Caribbean Territories - Montserrat,[0],09/26/2023
9,Nicaragua,[0],09/26/2023


In [96]:
my_dict_8= {}

In [97]:
slice_list[7]

['Micronesia (Federated States of)',
 'Malawi',
 'Monaco',
 'Bhutan',
 'Canada',
 'Burundi',
 'Hungary',
 'United Republic of Tanzania',
 'Nauru']

In [98]:
for country in slice_list[7]:
    print(country)
    time.sleep(30)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(30)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_8[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_8[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(30)



Micronesia (Federated States of)
country been selected successfully!
No warnings found for this country.
-----
Malawi
country been selected successfully!
Drought/Dry spell
Flood
Fog
Heat wave
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Tropical cyclone
Wind
Yes
No
Yes
48 hours
Yes
Yes
Yes
Yes
Yes
25,000 approximate
Drought/Dry spell
Flood
Fog
Heat wave
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Tropical cyclone
Wind
Yes
No
Yes
48 hours
Yes
Yes
Yes
Yes
Yes
25,000 approximate
-----
Monaco
country been selected successfully!
No warnings found for this country.
-----
Bhutan
country been selected successfully!
Flood
Landslide/Mudslide & Debris flow
Rain/Wet Spell
Snow
Tropical cyclone
Wind
Yes
No
Yes
No
Yes
Yes
Yes
No
Flood/GLOF early warning systems are set by basin not based on the Administrative boundaries
Flood
Landslide/Mudslide & Debris flow
Rain/Wet Spell
Snow
Tropical cyclone
Wind
Yes
No
Yes
No
Yes
Yes
Yes
No
Flood/GLOF early warning systems are set by basin not bas

In [99]:
my_dict_8

{'Micronesia (Federated States of)': [0],
 'Malawi': ['Drought/Dry spell',
  'Flood',
  'Fog',
  'Heat wave',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/Squall lines',
  'Tropical cyclone',
  'Wind',
  'Yes',
  'No',
  'Yes',
  '48 hours',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  '25,000 approximate'],
 'Monaco': [0],
 'Bhutan': ['Flood',
  'Landslide/Mudslide & Debris flow',
  'Rain/Wet Spell',
  'Snow',
  'Tropical cyclone',
  'Wind',
  'Yes',
  'No',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Flood/GLOF early warning systems are set by basin not based on the Administrative boundaries'],
 'Canada': ['Biomass burning plumes',
  'Cold wave',
  'Extra-tropical cyclone',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'High UV radiation',
  'Icing',
  'Lightning',
  'Pollen pollution/Polluted air',
  'Rain/Wet Spell',
  'Yes',
  'Yes',
  'Yes',
  '• For Hurricane Watches, Tropical Storm Watches, 

In [100]:
countries, warnings = zip(*my_dict_8.items())
df4 = pd.DataFrame({'Country': countries, 'Warnings': warnings})
df4

,Country,Warnings
0,Micronesia (Federated States of),[0]
1,Malawi,"[Drought/Dry spell, Flood, Fog, Heat wave, Lig..."
2,Monaco,[0]
3,Bhutan,"[Flood, Landslide/Mudslide & Debris flow, Rain..."
4,Canada,"[Biomass burning plumes, Cold wave, Extra-trop..."
5,Burundi,"[Flood, Rain/Wet Spell, No, Yes, No, No, Yes, ..."
6,Hungary,"[Cold wave, Flood, Fog, Freezing rain, Frost, ..."
7,United Republic of Tanzania,"[Cold wave, Drought/Dry spell, Dust storm/Sand..."
8,Nauru,[0]


In [101]:
df_loaded = pd.concat([df_loaded, df4], ignore_index=True)
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
...,...,...,...
64,Canada,"[Biomass burning plumes, Cold wave, Extra-trop...",NaN
65,Burundi,"[Flood, Rain/Wet Spell, No, Yes, No, No, Yes, ...",NaN
66,Hungary,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",NaN
67,United Republic of Tanzania,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",NaN


In [102]:
df_loaded.to_pickle('dataframe.pkl')
df_loaded = pd.read_pickle('dataframe.pkl')
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
...,...,...,...
64,Canada,"[Biomass burning plumes, Cold wave, Extra-trop...",NaN
65,Burundi,"[Flood, Rain/Wet Spell, No, Yes, No, No, Yes, ...",NaN
66,Hungary,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",NaN
67,United Republic of Tanzania,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",NaN


In [103]:
# Replace NaN values with the date
df_loaded["date"].fillna('09/27/2023', inplace=True)

In [104]:
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
...,...,...,...
64,Canada,"[Biomass burning plumes, Cold wave, Extra-trop...",09/27/2023
65,Burundi,"[Flood, Rain/Wet Spell, No, Yes, No, No, Yes, ...",09/27/2023
66,Hungary,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/27/2023
67,United Republic of Tanzania,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/27/2023


In [113]:
my_dict_9= {}

In [114]:
slice_list[8]


['Greece',
 'Germany',
 'Colombia',
 'France',
 'Andorra',
 'Austria',
 'Cyprus',
 'Finland',
 'Jordan']

In [115]:
for country in slice_list[8]:
    print(country)
    time.sleep(40)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(40)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_9[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_9[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
        break



Greece
country been selected successfully!
Cold wave
Dust storm/Sandstorm
Extra-tropical cyclone
Fog
Freezing rain
Frost
Hail
Haze/Smoke
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Yes
No
Yes
FROM LESS THAN 24 HOURS TO 2 DAYS
Yes
Yes
No
No
Cold wave
Dust storm/Sandstorm
Extra-tropical cyclone
Fog
Freezing rain
Frost
Hail
Haze/Smoke
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Yes
No
Yes
FROM LESS THAN 24 HOURS TO 2 DAYS
Yes
Yes
No
No
-----
Germany
country been selected successfully!
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Tornado
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
No
90000
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Tornado
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
No
90000
-----
Colombia
country been selected successfully!
Avalanche
Drought/Dry spell
Extra-tropical cyclone
Flood
Fog
F

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=117.0.5938.92)
Stacktrace:
0   chromedriver                        0x0000000109d2ded8 chromedriver + 5091032
1   chromedriver                        0x0000000109d24c43 chromedriver + 5053507
2   chromedriver                        0x00000001098c0366 chromedriver + 447334
3   chromedriver                        0x0000000109904fd1 chromedriver + 729041
4   chromedriver                        0x0000000109936662 chromedriver + 931426
5   chromedriver                        0x0000000109902ba8 chromedriver + 719784
6   chromedriver                        0x000000010993681e chromedriver + 931870
7   chromedriver                        0x00000001099526e1 chromedriver + 1046241
8   chromedriver                        0x0000000109936433 chromedriver + 930867
9   chromedriver                        0x0000000109901042 chromedriver + 712770
10  chromedriver                        0x000000010990226e chromedriv

In [116]:
my_dict_9

{'Greece': ['Cold wave',
  'Dust storm/Sandstorm',
  'Extra-tropical cyclone',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Yes',
  'No',
  'Yes',
  'FROM LESS THAN 24 HOURS TO 2 DAYS',
  'Yes',
  'Yes',
  'No',
  'No'],
 'Germany': ['Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Tornado',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'No',
  '90000'],
 'Colombia': ['Avalanche',
  'Drought/Dry spell',
  'Extra-tropical cyclone',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'High UV radiation',
  'Icing',
  'Lightning',
  'Pollen pollution/Polluted air',
  'Rain/Wet Spell',
  'Yes',
  'Yes',
  'Yes',
  '6 horas',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes

In [118]:
slice_list[8][4]

'Andorra'

In [120]:
for country in slice_list[8][4:]:
    print(country)
    time.sleep(40)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(40)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_9[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_9[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
        break



Andorra
country been selected successfully!
No warnings found for this country.
-----
Austria
country been selected successfully!
Avalanche
Cold wave
Drought/Dry spell
Freezing rain
Frost
Hail
Heat wave
High UV radiation
Lightning
Pollen pollution/Polluted air
Rain/Wet Spell
Snow
Snowstorm
Space weather event
Thunderstorms/Squall lines
Yes
Yes
Yes
5 days
Yes
Yes
Yes
Yes
Yes
100.000
Avalanche
Cold wave
Drought/Dry spell
Freezing rain
Frost
Hail
Heat wave
High UV radiation
Lightning
Pollen pollution/Polluted air
Rain/Wet Spell
Snow
Snowstorm
Space weather event
Thunderstorms/Squall lines
Yes
Yes
Yes
5 days
Yes
Yes
Yes
Yes
Yes
100.000
-----
Cyprus
Failed to select the country!


In [122]:
slice_list[8][6:]

['Cyprus', 'Finland', 'Jordan']

In [129]:
my_dict_9

{'Greece': ['Cold wave',
  'Dust storm/Sandstorm',
  'Extra-tropical cyclone',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Yes',
  'No',
  'Yes',
  'FROM LESS THAN 24 HOURS TO 2 DAYS',
  'Yes',
  'Yes',
  'No',
  'No'],
 'Germany': ['Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Tornado',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'No',
  '90000'],
 'Colombia': ['Avalanche',
  'Drought/Dry spell',
  'Extra-tropical cyclone',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'High UV radiation',
  'Icing',
  'Lightning',
  'Pollen pollution/Polluted air',
  'Rain/Wet Spell',
  'Yes',
  'Yes',
  'Yes',
  '6 horas',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes

In [125]:
for country in slice_list[8][6:]:
    print(country)
    time.sleep(40)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(40)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_9[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_9[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
        break



Cyprus
country been selected successfully!
Fog
Frost
Hail
Heat wave
High Seas/Rogue waves etc.
Icing
Rain/Wet Spell
Thunderstorms/Squall lines
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
No
Fog
Frost
Hail
Heat wave
High Seas/Rogue waves etc.
Icing
Rain/Wet Spell
Thunderstorms/Squall lines
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
No
-----
Finland
Failed to select the country!


In [131]:
slice_list[8][7:]

['Finland', 'Jordan']

In [132]:
for country in slice_list[8][7:]:
    print(country)
    time.sleep(50)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(50)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_9[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_9[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
        break



Finland
country been selected successfully!
Biomass burning plumes
Extra-tropical cyclone
Flood
Frost
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Rain/Wet Spell
Snow
Snowstorm
Space weather event
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tornado
Yes
Yes
Yes
5 days
No
Yes
Yes
No
I would say 100,000 (100). There are a plenty of different distribution channels: www-pages, radio, mobile apps, social media, traditional media etc..
Biomass burning plumes
Extra-tropical cyclone
Flood
Frost
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Rain/Wet Spell
Snow
Snowstorm
Space weather event
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tornado
Yes
Yes
Yes
5 days
No
Yes
Yes
No
I would say 100,000 (100). There are a plenty of different distribution channels: www-pages, radio, mobile apps, social media, traditional media etc..
-----
Jordan
country been selected successfully!
No warnings found for this country.
-----


In [134]:
slice_list[8]

['Greece',
 'Germany',
 'Colombia',
 'France',
 'Andorra',
 'Austria',
 'Cyprus',
 'Finland',
 'Jordan']

In [133]:
my_dict_9.keys()

dict_keys(['Greece', 'Germany', 'Colombia', 'France', 'Andorra', 'Austria', 'Cyprus', 'Finland', 'Jordan'])

In [135]:
countries, warnings = zip(*my_dict_9.items())
df5 = pd.DataFrame({'Country': countries, 'Warnings': warnings})
df5

,Country,Warnings
0,Greece,"[Cold wave, Dust storm/Sandstorm, Extra-tropic..."
1,Germany,"[Fog, Freezing rain, Frost, Hail, Heat wave, I..."
2,Colombia,"[Avalanche, Drought/Dry spell, Extra-tropical ..."
3,France,"[Avalanche, Cold wave, Drought/Dry spell, Extr..."
4,Andorra,[0]
5,Austria,"[Avalanche, Cold wave, Drought/Dry spell, Free..."
6,Cyprus,"[Fog, Frost, Hail, Heat wave, High Seas/Rogue ..."
7,Finland,"[Biomass burning plumes, Extra-tropical cyclon..."
8,Jordan,[0]


In [137]:
df5['date'] = '09/27/2023'

In [138]:
df5

,Country,Warnings,date
0,Greece,"[Cold wave, Dust storm/Sandstorm, Extra-tropic...",09/27/2023
1,Germany,"[Fog, Freezing rain, Frost, Hail, Heat wave, I...",09/27/2023
2,Colombia,"[Avalanche, Drought/Dry spell, Extra-tropical ...",09/27/2023
3,France,"[Avalanche, Cold wave, Drought/Dry spell, Extr...",09/27/2023
4,Andorra,[0],09/27/2023
5,Austria,"[Avalanche, Cold wave, Drought/Dry spell, Free...",09/27/2023
6,Cyprus,"[Fog, Frost, Hail, Heat wave, High Seas/Rogue ...",09/27/2023
7,Finland,"[Biomass burning plumes, Extra-tropical cyclon...",09/27/2023
8,Jordan,[0],09/27/2023


In [136]:
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
...,...,...,...
64,Canada,"[Biomass burning plumes, Cold wave, Extra-trop...",09/27/2023
65,Burundi,"[Flood, Rain/Wet Spell, No, Yes, No, No, Yes, ...",09/27/2023
66,Hungary,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/27/2023
67,United Republic of Tanzania,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/27/2023


In [139]:
df_loaded = pd.concat([df_loaded, df5], ignore_index=True)
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
...,...,...,...
73,Andorra,[0],09/27/2023
74,Austria,"[Avalanche, Cold wave, Drought/Dry spell, Free...",09/27/2023
75,Cyprus,"[Fog, Frost, Hail, Heat wave, High Seas/Rogue ...",09/27/2023
76,Finland,"[Biomass burning plumes, Extra-tropical cyclon...",09/27/2023


In [140]:
df_loaded.to_pickle('dataframe.pkl')
df_loaded = pd.read_pickle('dataframe.pkl')
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
...,...,...,...
73,Andorra,[0],09/27/2023
74,Austria,"[Avalanche, Cold wave, Drought/Dry spell, Free...",09/27/2023
75,Cyprus,"[Fog, Frost, Hail, Heat wave, High Seas/Rogue ...",09/27/2023
76,Finland,"[Biomass burning plumes, Extra-tropical cyclon...",09/27/2023


In [152]:
my_dict_10= {}

In [143]:
slice_list[9]

['Comoros',
 'Maldives',
 'Togo',
 'Luxembourg',
 'Bosnia and Herzegovina',
 'Central African Republic',
 'Myanmar',
 'Cabo Verde',
 'Bahamas']

In [144]:
for country in slice_list[9]:
    print(country)
    time.sleep(50)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(50)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_9[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_9[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
        break



Comoros
country been selected successfully!
Extra-tropical cyclone
Rain/Wet Spell
Tropical cyclone
Tsunami
Wind
No
No
Yes
No
Yes
Yes
No
No
No
50 000 personnes.
Extra-tropical cyclone
Rain/Wet Spell
Tropical cyclone
Tsunami
Wind
No
No
Yes
No
Yes
Yes
No
No
No
50 000 personnes.
-----
Maldives
country been selected successfully!
No warnings found for this country.
-----
Togo
country been selected successfully!
Drought/Dry spell
Dust storm/Sandstorm
Fog
Haze/Smoke
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Tornado
Wind
Yes
No
No
No
Yes
Yes
No
No
Pas de feedback
Drought/Dry spell
Dust storm/Sandstorm
Fog
Haze/Smoke
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Tornado
Wind
Yes
No
No
No
Yes
Yes
No
No
Pas de feedback
-----
Luxembourg
country been selected successfully!
No warnings found for this country.
-----
Bosnia and Herzegovina
country been selected successfully!
Cold wave
Drought/Dry spell
Flood
Fog
Freezing rain
Frost
Hail
Heat wave
High UV radiation
Lightning
Rain/Wet Sp

In [147]:
my_dict_9.keys()

dict_keys(['Greece', 'Germany', 'Colombia', 'France', 'Andorra', 'Austria', 'Cyprus', 'Finland', 'Jordan', 'Comoros', 'Maldives', 'Togo', 'Luxembourg', 'Bosnia and Herzegovina', 'Central African Republic', 'Myanmar', 'Cabo Verde', 'Bahamas'])

In [153]:
my_dict_10 = {k: my_dict_9[k] for k in list(my_dict_9.keys())[9:]}


In [154]:
my_dict_10.keys()

dict_keys(['Comoros', 'Maldives', 'Togo', 'Luxembourg', 'Bosnia and Herzegovina', 'Central African Republic', 'Myanmar', 'Cabo Verde', 'Bahamas'])

In [155]:
my_dict_10

{'Comoros': ['Extra-tropical cyclone',
  'Rain/Wet Spell',
  'Tropical cyclone',
  'Tsunami',
  'Wind',
  'No',
  'No',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'No',
  'No',
  'No',
  '50 000 personnes.'],
 'Maldives': [0],
 'Togo': ['Drought/Dry spell',
  'Dust storm/Sandstorm',
  'Fog',
  'Haze/Smoke',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/Squall lines',
  'Tornado',
  'Wind',
  'Yes',
  'No',
  'No',
  'No',
  'Yes',
  'Yes',
  'No',
  'No',
  'Pas de feedback'],
 'Luxembourg': [0],
 'Bosnia and Herzegovina': ['Cold wave',
  'Drought/Dry spell',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'High UV radiation',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wind',
  'No',
  'Yes',
  'Yes',
  '2 days',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'No',
  'Yes',
  'No',
  'the entire territory of the entity has 1,000,000 inhabitants who are covered by a warning system through the mechanism of th

In [156]:
countries, warnings = zip(*my_dict_10.items())
df6 = pd.DataFrame({'Country': countries, 'Warnings': warnings})
df6

,Country,Warnings
0,Comoros,"[Extra-tropical cyclone, Rain/Wet Spell, Tropi..."
1,Maldives,[0]
2,Togo,"[Drought/Dry spell, Dust storm/Sandstorm, Fog,..."
3,Luxembourg,[0]
4,Bosnia and Herzegovina,"[Cold wave, Drought/Dry spell, Flood, Fog, Fre..."
5,Central African Republic,"[Cold wave, Drought/Dry spell, Dust storm/Sand..."
6,Myanmar,"[Cold wave, Extra-tropical cyclone, Flood, Fog..."
7,Cabo Verde,"[Drought/Dry spell, Dust storm/Sandstorm, Fog,..."
8,Bahamas,[0]


In [157]:
df6['date'] = '09/27/2023'

In [158]:
df_loaded = pd.concat([df_loaded, df6], ignore_index=True)
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
...,...,...,...
82,Bosnia and Herzegovina,"[Cold wave, Drought/Dry spell, Flood, Fog, Fre...",09/27/2023
83,Central African Republic,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/27/2023
84,Myanmar,"[Cold wave, Extra-tropical cyclone, Flood, Fog...",09/27/2023
85,Cabo Verde,"[Drought/Dry spell, Dust storm/Sandstorm, Fog,...",09/27/2023


In [159]:
df_loaded.to_pickle('dataframe.pkl')
df_loaded = pd.read_pickle('dataframe.pkl')
df_loaded

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
...,...,...,...
82,Bosnia and Herzegovina,"[Cold wave, Drought/Dry spell, Flood, Fog, Fre...",09/27/2023
83,Central African Republic,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/27/2023
84,Myanmar,"[Cold wave, Extra-tropical cyclone, Flood, Fog...",09/27/2023
85,Cabo Verde,"[Drought/Dry spell, Dust storm/Sandstorm, Fog,...",09/27/2023


In [160]:
my_dict_11= {}

In [161]:
slice_list[10]

['Suriname',
 'Argentina',
 'French Polynesia',
 'Peru',
 'Kenya',
 'Sierra Leone',
 'Mozambique',
 'British Caribbean Territories - British Virgin Islands',
 'Republic of Moldova']

In [ ]:
slice_list[8][3]

In [ ]:
for country in slice_list[7]:
    print(country)
    time.sleep(30)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(30)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_8[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_8[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(30)



In [ ]:
test_list = ['Solomon Islands', 'Ireland', 'Romania', 'Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco', 'United States of America']
print(len(test_list))
my_dict = {}

In [ ]:
for country in test_list:
    print(country)
    time.sleep(5)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(5)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(10)



10 country

In [ ]:
test_list = [ 'Solomon Islands', 'Ireland', 'Romania', 'Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco', 'United States of America', 'Cyprus']
print(len(test_list))
my_dict = {}

In [ ]:
for country in test_list:
    print(country)
    time.sleep(12)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(5)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(10)

## 9 adjusting so we can run the full loop

In [ ]:
test_list = ['Solomon Islands', 'Ireland', 'Romania', 'Guyana']
print(len(test_list))
my_dict = {}

In [ ]:
success = select_country_from_dropdown(driver_instance, wait, "Benin")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

In [ ]:
success = select_country_from_dropdown(driver_instance, wait, country)

In [ ]:
for country in test_list:
    print(country)
    success = select_country_from_dropdown(driver_instance, wait, country)
    if success:
        print("country been selected successfully!")
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
    else:
        print("Failed to select the country!")
        


In [49]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException

def select_country(driver, wait, target_country):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    # Wait for the search input to be visible and interactable
    search_input = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.searchInput'))
    )

    # Clear the search input
    try:
        search_input.clear()
    except ElementNotInteractableException:
        # If the clear method fails, use JavaScript to set its value
        driver.execute_script("arguments[0].value = '';", search_input)

    loaded_countries = set()
    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll to the last element to load more countries
        ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()

    # After loading all countries, search for the target country and click on it
    for option in current_countries_elements:
        if option.text == target_country:
            option.click()
            break

    # Close the dropdown menu by clicking the dropdown button again
    dropdown.click()

    return loaded_countries


In [52]:
success = select_country_from_dropdown(driver_instance, wait, "Benin")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=117.0.5938.92)
Stacktrace:
0   chromedriver                        0x0000000108eaced8 chromedriver + 5091032
1   chromedriver                        0x0000000108ea3c43 chromedriver + 5053507
2   chromedriver                        0x0000000108a3f366 chromedriver + 447334
3   chromedriver                        0x0000000108a83fd1 chromedriver + 729041
4   chromedriver                        0x0000000108ab5662 chromedriver + 931426
5   chromedriver                        0x0000000108a81ba8 chromedriver + 719784
6   chromedriver                        0x0000000108ab581e chromedriver + 931870
7   chromedriver                        0x0000000108ad16e1 chromedriver + 1046241
8   chromedriver                        0x0000000108ab5433 chromedriver + 930867
9   chromedriver                        0x0000000108a80042 chromedriver + 712770
10  chromedriver                        0x0000000108a8126e chromedriv